## IMPORTS

In [83]:
import pandas as pd
import inflection
import math

### Helper functions

### Loading data

In [84]:
df_sales_raw=pd.read_csv('Datasets/train.csv', low_memory=False)
df_store_raw=pd.read_csv('Datasets/store.csv', low_memory=False)

# merge

df_raw=pd.merge(df_sales_raw,df_store_raw, how='left', on='Store')

In [85]:
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
167777,528,2,2015-03-03,14057,1066,1,1,0,0,a,c,20620.0,NaN,NaN,1,37.0,2009.0,"Jan,Apr,Jul,Oct"


## DATA DESCRIPTION

In [86]:
df1=df_raw.copy()
df1.columns


Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

### Rename columns

In [87]:
cols_old=['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

snakecase=lambda x: inflection.underscore(x)

cols_new=list(map(snakecase,cols_old))

#rename 
df1.columns=cols_new

### Data dimensions

In [88]:
print('Numbers of raw:{}'.format(df1.shape[0]))
print('Numbers of cols:{}'.format(df1.shape[1]))

Numbers of raw:1017209
Numbers of cols:18


### Data Type

In [89]:
df1['date']=pd.to_datetime(df1['date'])

df1.dtypes


store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

### Check NA

In [90]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

### Fillout NA

In [91]:
df1['competition_distance'].max()

75860.0

In [95]:
#competition_distance              
df1['competition_distance']=df1['competition_distance'].apply(lambda x:200000.0 if math.isnan(x) else x)

#competition_open_since_month  
df1['competition_open_since_month']=df1.apply(lambda x: x['date'].month 
                                              if math.isnan(x['competition_open_since_month']) 
                                              else x['competition_open_since_month'], axis=1)

#competition_open_since_year  
df1['competition_open_since_year']=df1.apply(lambda x: x['date'].year 
                                              if math.isnan(x['competition_open_since_year']) 
                                              else x['competition_open_since_year'], axis=1)

#promo2_since_week  
df1['promo2_since_week']=df1.apply(lambda x: x['date'].week 
                                              if math.isnan(x['promo2_since_week']) 
                                              else x['promo2_since_week'], axis=1)
#promo2_since_year 
df1['promo2_since_year']=df1.apply(lambda x: x['date'].year 
                                              if math.isnan(x['promo2_since_year' ]) 
                                              else x['promo2_since_year'], axis=1)
#promo_interval   
month_map={1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
df1['promo_interval'].fillna(0,inplace=True)
df1['month_map']=df1['date'].dt.month.map(month_map)
df1['is_promo']=df1[['promo_interval','month_map']].apply(
    lambda x:0 if x['promo_interval']==0 else 1 if x['month_map'] in x['promo_interval'].split(',')else 0, axis=1)

In [96]:
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64